# Data Augmentation w/ Synthetic JSON & Synthetic Biographies Using OpenAI and Distilabel (the code to generate synthetic biographies from JSON is in 2 other files)



### **Promlem Statement & Solution**
**Structured Information Extraction (IE)** at the document level is an exceptionally challenging task, especially when the input documents are long and the required JSON outputs are nested. **Manual data collection**, while accurate, also consumes a significant amount of time.

These lead to 2 implications:
1.   Structured data on elites - which is required for large-scale construction of elites' professional, familial, and education peer networks - is extremely limited
2.   Training data for automated structured IE are also very scant.

**Solution:** To address this challenge of data scarcity, this pipeline create a synthetic dataset for fine-tuning LLMs for structured IE task from a small amount of data .

<u>The objective</u>: Using LLMs and human correction to create a dataset containing pairs of biographical documents & corresponding JSON outputs for fine-tuning the appropriate LLMs for structured IE.

Specifically, the following workflow gathers a small number of strutured JSON records of elites' education, career, and family details and then use them to *generate (1) additional synthetic JSONs, (2) synthetic biographies, and (3) paraphrased biographies.*

This is to ensure that the fine-tuning data contain a diverse set of examples, with both original and synthetic JSONs as well as different (paraphrased) version of the factually correct biographies.



### **Process Overview**
**Original data:** 102 JSON records of real, factual political elites in *Vietnam* & *Russia*

**The strategy contains several steps**
1. Generate synthetic 102 JSON outputs based on actual JSON outputs
    * <u>Toolset</u>: Programming to randomly draw from a given list of names to replace the names in the original JSONs
    * <u>Justification</u>: Only mechanical swapping (from a list of first, middle, and last names) is used for synthetic JSONs. No LLM is used to avoid unnecessary hallucinations.
    * <u>Result</u>: *204 JSONs* (102 original + 102 fictional)

2. Manual correction & enhancement of JSONs by human

3. Generate synthetic biographies based on all JSON records (both original & synthetic JSONs)
    * <u>Toolset</u>: GPT-4-Turbo, Llama 3.3-70B & Distilabel
    * Justification: 2 LLMs are used to create a diversified pool of synthetic biographies for human evaluation and selection
    * <u>Result</u>: *204 biographies* (both factual & fictional)

4. Manual selection (to pick the best between synthetic biographies by Llama 3.3 70B and GPT-4-Turbo), correction, and enhancement of synthetic biographies by human.

5. For each biography of the original JSONs, generate another version of bio document with paraphrasing & rewriting
    * <u>Toolset</u>: Llama 3.3-70B & Distilabel
    * <u>Result</u>: *102 additional* (paraphrased) biographies for the original JSONs

6. Final manual check & enhancement


**FINAL PRODUCT:** Total of *306 pairs of biography-JSON*
* Factual JSON & biographies: 102
* Factual JSON & (paraphrased) biographies: 102
* Fictional JSON & biographies: 102



### Installations

In [ ]:
!pip install distilabel pydantic torch openpyxl # llama-cpp-python

### Load original data & define Pydantic model

In [ ]:
import json
import pandas as pd
from pydantic import BaseModel
import typing # Import the typing module
from typing import List, Optional

# Load the JSONL file - file named *final3.jsonl
file_path = "/content/drive/llm_train_2025.03.16.xlsx"
df = pd.read_excel(file_path)

In [ ]:
df.head()

,instruction,input,output
0,You are an expert historian and political scie...,"Bùi Danh Lưu, also known by his other name Quo...","{""Name"": ""Bui Danh Luu"", ""Name_Other"": [""Quoc ..."
1,You are an expert historian and political scie...,Bui Thien Ngo\n================\n\nBui Thien N...,"{""Name"": ""Bui Thien Ngo"", ""Name_Other"": [], ""B..."
2,You are an expert historian and political scie...,Cao Si Kiem is a Vietnamese economist and poli...,"{""Name"": ""Cao Si Kiem"", ""Name_Other"": [""Cao Sy..."
3,You are an expert historian and political scie...,Chu Tuan Nha is a former Vietnamese politician...,"{""Name"": ""Chu Tuan Nha"", ""Name_Other"": [], ""Bi..."
4,You are an expert historian and political scie...,Dam Quang Trung (1921-1995) was a general in t...,"{""Name"": ""Dam Quang Trung"", ""Name_Other"": [""Da..."


In [ ]:
import json
import pandas as pd
from pydantic import BaseModel, Field
import typing # Import the typing module
from typing import List, Optional

class Birthloc(BaseModel):
    City: str = ""
    City_OtherName: List[str] = []
    Region: str = ""
    Region_OtherName: List[str] = []

class Edu(BaseModel):
    School: str = ""
    OtherName: List[str] = []
    Start: str = ""
    End: str = ""
    Level: str = ""
    Field: str = ""
    Location: List[str] = []  # ✅ Now correctly set as a list

class Military_Title(BaseModel):
    Title: str = ""
    YearReceived: str = ""

class Org(BaseModel):
    MainOrg: str = ""
    MainOrg_OtherName: str = ""
    SubOrg: str = ""
    Unit: str = ""

class JobModel(BaseModel):  # Renamed to Job
    Start: str = ""
    End: str = ""
    Position: str = ""
    Organization: Org = Org()  # Ensures default structure
    Location: List[str] = []

class CareerEvent(BaseModel):
    Date: str = ""
    Detail: List[str] = []

class FamRelation(BaseModel):
    Name: str = ""
    Relation: str = ""

class Coworker(BaseModel):
    FirstMention_Year: str = ""
    Name: List[str] = []

class Opponent(BaseModel):
    FirstMention_Year: str = ""
    Name: List[str] = []

class Biography(BaseModel):
    Name: str = ""
    Name_Other: List[str] = []
    BirthYear: str = ""
    DeathYear: str = ""
    Birthplace: Birthloc = Birthloc()  # Ensures default structure
    Ethnicity: List[str] = []
    Education: List[Edu] = [Edu()]  # Ensures at least one empty object
    MilitaryTitle: List[Military_Title] = [Military_Title()]  # Ensures at least one empty object
    Job: List[JobModel] = [JobModel()]  # Ensures at least one empty object
    Retired: List[CareerEvent] = [CareerEvent()]
    Dismissed: List[CareerEvent] = [CareerEvent()]
    Resigned: List[CareerEvent] = [CareerEvent()]
    Arrested: List[CareerEvent] = [CareerEvent()]
    Exiled: List[CareerEvent] = [CareerEvent()]
    Killed: List[CareerEvent] = [CareerEvent()]
    FamilyMember: List[FamRelation] = [FamRelation()]
    Colleague: List[Coworker] = [Coworker()]
    Rival: List[Opponent] = [Opponent()]


### Generate synthetic JSON outputs through swapping

In [ ]:
### Creating Lists & Defining Necessary Functions ###
import pandas as pd
import json
import random

# Separate lists of Vietnamese and Russian names for synthetic data generation
vietnamese_first_names = ["An", "Cuong", "Dung", "Hai", "Khanh", "Minh", "Nam", "Quang", "Tuan"]
vietnamese_middle_names = ["Van", "Thi", "Huu", "Đinh", "Xuan", "Thanh", "Trong", "Ngoc", "The", "Ba", "Cong"]
vietnamese_last_names = ["Nguyen", "Tran", "Le", "Pham", "Hoang", "Vu", "Đang", "Bui", "Đo", "Ngo"]

russian_first_names = ["Ivan", "Dmitry", "Sergei", "Nikolai", "Viktor", "Yuri", "Oleg", "Boris", "Alexei", "Andrei"]
russian_middle_names = ["Alexandrovich", "Petrovich", "Vladimirovich", "Mikhailovich", "Sergeyevich", "Ivanovich", "Yurievich", "Nikitovich", "Anatolyevich", "Vasilievich"]
russian_last_names = ["Ivanov", "Petrov", "Sidorov", "Kuznetsov", "Smirnov", "Popov", "Volkov", "Sokolov", "Orlov", "Vasilyev"]

# Function to generate a new name based on nationality
def generate_fake_name(nationality="Vietnamese"):
    if nationality == "Vietnamese":
        return f"{random.choice(vietnamese_last_names)} {random.choice(vietnamese_middle_names)} {random.choice(vietnamese_first_names)}"
    else:
        return f"{random.choice(russian_first_names)} {random.choice(russian_middle_names)} {random.choice(russian_last_names)}"

# Function to shift dates forward by 10 years
def shift_date(date_str):
    if not date_str or not isinstance(date_str, str):
        return date_str  # Return as is if empty or invalid

    parts = date_str.split()
    if len(parts) == 1:  # Year format YYYY
        try:
            return str(int(parts[0]) + 10)
        except ValueError:
            return date_str  # Return unchanged if not a valid number
    elif len(parts) == 2:  # Month Year format (e.g., March 1990)
        try:
            return f"{parts[0]} {int(parts[1]) + 10}"
        except ValueError:
            return date_str  # Return unchanged if parsing fails
    return date_str

# Function to update family names based on relation
def update_family_names(original_json, nationality):
    if "FamilyMember" not in original_json or not original_json["FamilyMember"]:
        return
    elite_family_name = original_json["Name"].split()[0] if nationality == "Vietnamese" else original_json["Name"].split()[-1]
    matching_relations = {"Grandfather", "Father", "Brother", "Sister",  "Son", "Daughter", "Grandson", "Granddaughter", "Niece", "Nephew"}
    for member in original_json["FamilyMember"]:
        if member["Relation"] in matching_relations and member["Name"]:
            name_parts = member["Name"].split()
            if nationality == "Vietnamese" and len(name_parts) > 1:
                member["Name"] = f"{elite_family_name} {' '.join(name_parts[1:])}"
            elif nationality == "Russian" and len(name_parts) > 1:
                member["Name"] = f"{' '.join(name_parts[:-1])} {elite_family_name}"

# Function to update Colleagues' and Rivals' names if they exist
def update_colleagues_and_rivals(original_json, nationality):
    if "Colleague" in original_json and original_json["Colleague"]:
        for colleague in original_json["Colleague"]:
            colleague["Name"] = [generate_fake_name(nationality) for _ in colleague["Name"]]
    if "Rival" in original_json and original_json["Rival"]:
        for rival in original_json["Rival"]:
            rival["Name"] = [generate_fake_name(nationality) for _ in rival["Name"]]

In [ ]:
### Executing the swapping process for the entire dataset ###

# Set seed for reproducibility
random.seed(1234)

# Process each JSON output and create synthetic versions with proper key checking
synthetic_jsons = []
for index, row in df.iterrows():
    try:
        original_json = json.loads(row["output"])  # Parse JSON string
        nationality = "Vietnamese" if index < 91 else "Russian"  # Determine nationality

        # Replace names
        new_name = generate_fake_name(nationality)
        original_json["Name"] = new_name

        # Update name in "Name_Other" if present
        if "Name_Other" in original_json:
            original_json["Name_Other"] = [generate_fake_name(nationality) for _ in original_json["Name_Other"]]

        # Shift all date fields by 10 years if they exist
        if "BirthYear" in original_json and original_json["BirthYear"]:
            original_json["BirthYear"] = shift_date(original_json["BirthYear"])
        if "DeathYear" in original_json and original_json["DeathYear"]:
            original_json["DeathYear"] = shift_date(original_json["DeathYear"])

        for edu in original_json.get("Education", []):
            if "Start" in edu and edu["Start"]:
                edu["Start"] = shift_date(edu["Start"])
            if "End" in edu and edu["End"]:
                edu["End"] = shift_date(edu["End"])

        for job in original_json.get("Job", []):
            if "Start" in job and job["Start"]:
                job["Start"] = shift_date(job["Start"])
            if "End" in job and job["End"]:
                job["End"] = shift_date(job["End"])

        for military in original_json.get("MilitaryTitle", []):
            if "YearReceived" in military and military["YearReceived"]:
                military["YearReceived"] = shift_date(military["YearReceived"])

        for event_type in ["Retired", "Dismissed", "Resigned", "Arrested", "Exiled", "Killed"]:
            for event in original_json.get(event_type, []):
                if "Date" in event and event["Date"]:
                    event["Date"] = shift_date(event["Date"])

        for relation in ["Colleague", "Rival"]:
            for person in original_json.get(relation, []):
                if "FirstMention_Year" in person and person["FirstMention_Year"]:
                    person["FirstMention_Year"] = shift_date(person["FirstMention_Year"])

        update_family_names(original_json, nationality)
        update_colleagues_and_rivals(original_json, nationality)

        # Store synthetic JSON
        synthetic_jsons.append(original_json)

    except json.JSONDecodeError:
        print(f"Skipping row {index} due to JSON decoding error.")


In [ ]:
### Downloading the synthetic JSON records ###
# Convert to DataFrame
synthetic_df = pd.DataFrame({"synthetic_output": [json.dumps(entry, ensure_ascii=False) for entry in synthetic_jsons]})

# Save the updated file with reproducibility
synth_file_path = "/content/drive/elitenet_synthetic_json.xlsx"
synthetic_df.to_excel(synth_file_path, index=False)


**After this, the synthetic JSONs are downloaded to be processed and augmented by human --> increased quality**

### Loading the enhanced synthetic JSON data (after human correction & augmentation)

In [ ]:
import os
import json
import time
import pandas as pd
import openai  # ✅ OpenAI API for GPT-4 Turbo

# 📌 Load Data
df = pd.read_excel("/content/drive/elitenet_synthetic_json_final.xlsx")  # Replace with actual file path

### Generating Biographies (from both original & fictional JSONs): 2 LLMs
**2 LLMs:** OpenAI (GPT-4 Turbo), Llama

**Framework:** Distilabel
 * Distilabel is a framework by Argilla team for fast and reliable synthetic data generation  

In [ ]:
import distilabel
print(distilabel.__version__)

1.5.3


#### Generator 1: GPT-4 Turbo

In [ ]:
# Initialize & load model
from distilabel.models.llms import OpenAILLM

chatgpt = OpenAILLM(
    model="gpt-4-turbo",
    api_key="yourOpenAI-APIkey"
    )

chatgpt.load()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │         async_ = False                                                                       │ │
│ │       code_obj = <code object <cell line: 0> at 0x7fbdf2169110, file                         │ │
│ │                  "<ipython-input-5-3429efc429e7>", line 1>                                   │ │
│ │ old_excepthook = <bound method IPKernelApp.excepthook of <__main__.ColabKernelApp object at  │ │
│ │                  0x7fbe52d65d90>>                                                            │ │
│ │        outflag = True                                                                        │ │
│ │         result = <ExecutionResult object at 7fbdf4e87f90, execution_count=5                  │ │
│ │                  error_before_exec=None error_in_exec=`instructor` is not installed. Please  │ │
│ │                  install it using `pip install 'distilabel[instructor]'`.                    │ │
│ │                  info=<ExecutionInfo object at 7fbdf5f1ff10, raw_cell="# Initialize & load   │ │
│ │                  model                                                                       │ │
│ │                  from distilabel.models.l.." store_history=True silent=False                 │ │
│ │                  shell_futures=True cell_id=None> result=None>                               │ │
│ │           self = <google.colab._shell.Shell object at 0x7fbe3dda2fd0>                        │ │
│ │             tb = <traceback object at 0x7fbdf1526500>                                        │ │
│ │          value = ImportError("`instructor` is not installed. Please install it using `pip    │ │
│ │                  install 'distilabel[instructor]'`.")                                        │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ in <cell line: 0>:10                                                                             │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │     chatgpt = OpenAILLM(                                                                     │ │
│ │               │   generation_kwargs={},                                                      │ │
│ │               │   use_offline_batch_generation=False,                                        │ │
│ │               │   offline_batch_generation_block_until_done=None,                            │ │
│ │               │   jobs_ids=None,                                                             │ │
│ │               │   model='gpt-4-turbo',                                                       │ │
│ │               │   base_url='https://api.openai.com/v1',                                      │ │
│ │               │   api_key=SecretStr('**********'),       

In [ ]:
!nvidia-smi

Mon Mar 17 11:26:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
torch.cuda.empty_cache()

##### Testing OpenAI's (GPT-4 Turbo) synthetic biography generation with just 1 JSON record

In [ ]:
### Test OpenAI for 1 JSON ###
def generate_with_openai(prompt, max_tokens=1500, temp=0.7):
    """Generate a biography using OpenAI GPT-4 Turbo via Distilabel."""
    try:
        response = chatgpt.generate_outputs(
            inputs=[[{"role": "user", "content": prompt}]],  # ✅ Use Distilabel format
            max_new_tokens=max_tokens,
            temperature=temp
        )

        if response and isinstance(response, list) and len(response) > 0:
            first_result = response[0]  # ✅ Extract first dictionary from list
            if isinstance(first_result, dict) and "generations" in first_result and first_result["generations"]:
                return first_result["generations"][0]  # ✅ Extract biography text

        print("❌ Distilabel returned an empty response.")
        return ""

    except Exception as e:
        print(f"❌ OpenAI via Distilabel Error: {e}")
        return ""


In [ ]:
import json
test_json = df["synthetic_output"][4]
test_data = json.loads(test_json)  # Convert string to dictionary
print(test_data)

{'Name': 'Nguyen The Quang', 'Name_Other': ['Pham Thi Tuan'], 'BirthYear': '1931', 'DeathYear': '2005', 'Birthplace': {'City': 'Ha Quang', 'City_OtherName': [], 'Region': 'Cao Bang', 'Region_OtherName': []}, 'Ethnicity': ['Tay'], 'Education': [{'School': 'Whampoa Military Academy', 'OtherName': [], 'Start': '1951', 'End': '1953', 'Level': '', 'Field': '', 'Location': ['China']}, {'School': 'Frunze Military Academy', 'OtherName': [], 'Start': '1967', 'End': '1967', 'Level': '', 'Field': '', 'Location': ['USSR']}], 'MilitaryTitle': [{'Title': 'Colonel', 'YearReceived': '1968'}, {'Title': 'Major General', 'YearReceived': '1984'}, {'Title': 'Lieutenant General', 'YearReceived': '1990'}, {'Title': 'Senior Lieutenant General', 'YearReceived': '1994'}], 'Job': [{'Start': '1951', 'End': '1953', 'Position': 'Student', 'Organization': {'MainOrg': 'Whampoa Military Academy', 'MainOrg_OtherName': '', 'SubOrg': '', 'Unit': ''}, 'Location': ['China']}, {'Start': '1954', 'End': '1955', 'Position': 'L

In [ ]:
test_prompt = f"""
You are an expert historian and writer specializing in biographies of political elites (these elites are either from Vietnam or Russia).
Generate a detailed biography based on the following elite's JSON record:

{json.dumps(test_data, indent=2)}

Important Notes:
- The biography should be **500-1200 words**, depending on the amount of available details of the JSON record.
- Write in a **neutral, Wikipedia-style format**, **without a References section**.
- When writing the biography, include **all available JSON details**, such as education, job experiences, dismissal, retirement, titles, family members, colleagues, and rivals. This essentially means copy-paste the all the avaialble content in the JSON fields (if they are not empty) into your sentences and paragraphs when you write the biography.
- **DO NOT fabricate missing details**. If certain sections or JSON fields are empty, omit missing information naturally and avoid mentioning them explicitly.
- **Enhance readability and coherence** while preserving facts.
    """

test_response = generate_with_openai(test_prompt)
print(test_response)

### Biography of Nguyen The Quang

Nguyen The Quang was a prominent military and political figure in Vietnam, born in 1931 in Ha Quang, Cao Bang. He was of Tay ethnicity, a significant minority group in Vietnam's northern region. Quang's career was marked by extensive military service and leadership within the Communist Party of Vietnam (CPV), shaping his legacy as a key figure in Vietnam's post-war era.

#### Early Life and Education

Nguyen The Quang embarked on his military education at the Whampoa Military Academy in China, attending from 1951 to 1953. This period was crucial as it laid the foundational military skills and ideological training that would define his career. He further advanced his military education at the prestigious Frunze Military Academy in the USSR for a brief period in 1967, gaining exposure to broader military strategies and strengthening international ties that were vital during the Cold War era.

#### Military Career

Quang's military career began earnestly

In [ ]:
test_prompt2 = """
You are an expert historian and writer specializing in biographies of political elites (Vietnam or Russia).
Your task is to generate a **detailed, Wikipedia-style biography** based on structured JSON data.

### **Example JSON Record & Biography**
Below is an **example JSON record** and its corresponding **expected biography**:

**Example JSON Record:**
{
  "Name": "To Huy Rua",
  "Name_Other": [],
  "BirthYear": "1947",
  "DeathYear": "",
  "Birthplace": {"City": "Quang Xuong", "City_OtherName": [], "Region": "Thanh Hoa",  "Region_OtherName": []},
  "Ethnicity": [],
  "Education": [
    {"School": "Central Propaganda School", "OtherName": ["Academy of Journalism and Communication"], "Start": "1970", "End": "1974", "Level": "", "Field": "", "Location": []},
    {"School": "Hanoi University of Science", "OtherName": ["Vietnam National University, Hanoi"], "Start": "1975", "End": "1979", "Level": "University", "Field": "Mathematics", "Location": ["Hanoi"]},
    {"School": "Academy of Social Sciences", "OtherName": [], "Start": "1980", "End": "1986", "Level": "Doctorate", "Field": "Philosophy", "Location": ["USSR"]}
      ],
  "MilitaryTitle": [{"Title": "", "YearReceived": ""}],
  "Job": [
    {"Start": "1965", "End": "1970", "Position": "", "Organization": {"MainOrg": "Youth Volunteer Force", "MainOrg_OtherName": "", "SubOrg": "", "Unit": ""}, "Location": ["South"]},
    {"Start": "1974", "End": "1979", "Position": "Teaching Assistant", "Organization": {"MainOrg": "Central Propaganda School", "MainOrg_OtherName": "Academy of Journalism and Communication", "SubOrg": "", "Unit": ""}, "Location": []},
    {"Start": "1986", "End": "", "Position": "Deputy Head", "Organization": {"MainOrg": "Hanoi University of Science", "MainOrg_OtherName": "Vietnam National University, Hanoi", "SubOrg": "Faculty of Philisophy", "Unit": ""}, "Location": ["Hanoi"]},
    {"Start": "1990", "End": "1995", "Position": "Director", "Organization": {"MainOrg": "Central Propaganda School", "MainOrg_OtherName": "Academy of Journalism and Communication", "SubOrg": "", "Unit": ""}, "Location": []},
    {"Start": "1996", "End": "", "Position": "Deputy Director", "Organization": {"MainOrg": "Ho Chi Minh National Academy of Politics", "MainOrg_OtherName": "Ho Chi Minh National Academy of Politics and Public Administration", "SubOrg": "", "Unit": ""}, "Location": []},
    {"Start": "2000", "End": "", "Position": "Secretary", "Organization": {"MainOrg": "City Party Committee", "MainOrg_OtherName": "", "SubOrg": "", "Unit": ""}, "Location": ["Hai Phong"]},
    {"Start": "2003", "End": "", "Position": "Deputy Director", "Organization": {"MainOrg": "Ho Chi Minh National Academy of Politics and Public Administration", "MainOrg_OtherName": "", "SubOrg": "", "Unit": ""}, "Location": []},
    {"Start": "2005", "End": "", "Position": "Director", "Organization": {"MainOrg": "Ho Chi Minh National Academy of Politics and Public Administration", "MainOrg_OtherName": "Ho Chi Minh National Academy of Politics", "SubOrg": "", "Unit": ""}, "Location": []},
    {"Start": "May 2006", "End": "", "Position": "Head", "Organization": {"MainOrg": "Communist Party of Vietnam", "MainOrg_OtherName": "CPV", "SubOrg": "Central Ideology and Culture Department", "Unit": ""}, "Location": []},
    {"Start": "May 2007", "End": "2010", "Position": "Chairman", "Organization": {"MainOrg": "CPV", "MainOrg_OtherName": "", "SubOrg": "Central Theoretical Council", "Unit": ""}, "Location": []},
    {"Start": "May 2007", "End": "2010", "Position": "Head", "Organization": {"MainOrg": "Communist Party of Vietnam", "MainOrg_OtherName": "CPV", "SubOrg": "Central Propaganda Department", "Unit": ""}, "Location": []},
    {"Start": "February 2011", "End": "", "Position": "Head", "Organization": {"MainOrg": "Communist Party of Vietnam", "MainOrg_OtherName": "CPV", "SubOrg": "Central Organization Committee", "Unit": ""}, "Location": []},
    ],
  "Retired": [{"Date": "January 2016", "Detail": ["At the 12th National Congress (January 2016) he retired."]}],
  "Dismissed": [{"Date": "", "Detail": []}],
  "Resigned": [{"Date": "", "Detail": []}],
  "Arrested": [{"Date": "", "Detail": []}],
  "Exiled": [{"Date": "", "Detail": []}],
  "Killed": [{"Date": "", "Detail": []}],
  "FamilyMember": [{"Name": "Truong Tuyet Nhung", "Relation": "Wife"}, {"Name": "To Tu Ha", "Relation": "Son"}, {"Name": "To Huy Vu", "Relation": "Son"}, {"Name": "To Linh Huong", "Relation": "Daughter"}],
  "Colleague": [{"FirstMention_Year": "1999", "Name": ["Le Kha Phieu"]}, {"FirstMention_Year": "", "Name": ["Pham Van Vung"]}],
  "Rival": [{"FirstMention_Year": "", "Name": []}]
}

**Expected Biography:**
To Huy Rua (born 1947 ) is a Vietnamese politician . He is currently the President of the Vietnam - Japan Friendship Association, Honorary President of Vietnam - Japan University , Hanoi National University . He was a former member of the Politburo of the Communist Party of Vietnam (CPV), Head of the Central Organization Committee , former Head of the Central Propaganda Department, Director of the Ho Chi Minh National Academy of Politics , and Director of the Central Propaganda School (now the Academy of Journalism and Propaganda ). He graduated with a Bachelor's degree in Mathematics from Hanoi University of Science , and a Doctorate in Philosophy (Soviet Academy of Sciences). [ 1 ]
Biography
He was born on June 4, 1947 , in Quang Thai commune, Quang Xuong district , Thanh Hoa province .
In 1965, he joined the Youth Volunteer Force to serve on the battlefield in the Southern region. In the early 1970, he was sent to study at the Central Propaganda School (now the Academy of Journalism and Communication), majoring in Philosophy. After graduating in 1974, he was retained as a teaching assistant at the Faculty of Philosophy at the school. During his time as a teaching assistant, he registered for additional studies (1975 – 1979) and earned a Bachelor's degree in Mathematics at Hanoi University of Science (now Vietnam National University, Hanoi).
In 1980, he was appointed as a postgraduate student, majoring in Philosophy at the Academy of Social Sciences of the USSR, successfully defending his doctoral thesis . After graduating in 1986, he returned to Vietnam and was appointed Deputy Head of the Faculty of Philosophy, Hanoi University of Science (now Vietnam National University, Hanoi).
From 1990 to 1995, he was elected Director of the Central Propaganda School (now the Academy of Journalism and Communication ). During this time, he was awarded the title of Meritorious Teacher by the Vietnamese State and the academic title of Associate Professor of Philosophy.
In 1996, he was appointed Deputy Director of the Ho Chi Minh National Academy of Politics (now known as the Ho Chi Minh National Academy of Politics and Public Administration).
At the end of 1999, he was appointed by his fellow countryman, General Secretary (of the CPV) Le Kha Phieu,  as Secretary of the Hai Phong City Party Committee and Secretary of the Hai Phong City Military Party Committee. He began this position in 2000.
In 2003, he was transferred back to work as Deputy Director of the Ho Chi Minh National Academy of Politics and Public Administration.
At the end of 2004, he was appointed Director of the Ho Chi Minh National Academy of Politics and Public Administration (also known as the Ho Chi Minh National Academy of Politics) and took charge of this position in 2005.
On May 1, 2006 , he was assigned as Head of the Central Ideology and Culture Department of the Communist Party of Vietnam (CPV) (now the Central Propaganda Department).
From May 8, 2007 to 2010, To Huy Rua was assigned as Chairman of the Central Theoretical Council (of the CPV) and Head of the Central Propaganda Department of the Communist Party of Vietnam (CPV) .
On February 7, 2011, he was appointed Head of the Central Organization Committee of the Communist Party of Vietnam (CPV).
At the 12th National Congress (January 2016) he retired.
Family
• His wife is Ms. Truong Tuyet Nhung, former Head of Training Department, Academy of Journalism and Propaganda.
•	First son: To Tu Ha, Director of Noi Bai International Airport.
•	Second son: To Huy Vu, currently Director of the International Cooperation Department of the State Bank of Vietnam.
•	Youngest daughter: To Linh Huong, born in 1988, studied journalism, was elected as Chairwoman of the Board of Directors of Vinaconex - PVC Joint Stock Company.
Controversial political activity
To Huy Rua was reported to having sponsored the rise of Pham Van Vung to be a Navy Political Commissar. When he became Navy Political Commissar, Pham Van Vung revealed his true colors as a corrupt official, and was forced to hand over his position after 1 year in office before deciding to retire.

---

### **Task: Now Generate a Biography for the Following JSON Record**

""".strip() + "\n\n" + json.dumps(test_data, indent=2) + "\n\n" + """

Important Notes:
- The biography should be **500-1200 words**, depending on the amount of available details of the JSON record.
- Write the biography in a **neutral, Wikipedia-style format**, **without a References section**.
- When writing the biography, include **all available JSON details**, such as education, job experiences, dismissal, retirement, titles, family members, colleagues, and rivals.
- **DO NOT fabricate missing details**. If certain sections or JSON fields are empty, omit missing information naturally and avoid mentioning them explicitly.
- **Enhance readability and coherence** while preserving facts.

    """

test_response2 = generate_with_openai(test_prompt)
print(test_response2)

Nguyen The Quang, born in 1931 in Ha Quang, Cao Bang, was a prominent Vietnamese military and political figure. He hailed from the Tay ethnicity, one of the many ethnic groups in Vietnam. Quang's military and political career spanned several decades, reflecting his significant role in Vietnam's modern history. He passed away in 2005.

Quang's early education occurred at the prestigious Whampoa Military Academy in China from 1951 to 1953, where he was a student during the early years of his career. This period was crucial for acquiring the military skills that later defined his career. He furthered his military education by attending the Frunze Military Academy in the USSR for a brief period in 1967.

Upon returning from Whampoa, Quang joined the Vietnam People's Army, quickly ascending through various leadership positions. From 1954 to 1955, he held a leadership role within the Army, and from 1955 to 1956, he served as the head of the Hanoi Special Zone. His responsibilities expanded a

##### Scaling this function for the entire JSON dataset (in batches)

In [ ]:
### Synthetic biography generation
BATCH_SIZE = 4  # Adjust based on rate limits
DEBUG = False

prompts, batch_indices, failed_indices = [], [], []

def generate_with_openai(prompt, max_tokens=1600, temp=0.65):
    """Generate a biography using OpenAI GPT-4 Turbo via Distilabel."""
    try:
        response = chatgpt.generate_outputs(
            inputs=[[{"role": "user", "content": prompt}]],  # ✅ Use Distilabel format
            max_new_tokens=max_tokens,
            temperature=temp
        )

        if response and isinstance(response, list) and len(response) > 0:
            first_result = response[0]  # ✅ Extract first dictionary from list
            if isinstance(first_result, dict) and "generations" in first_result and first_result["generations"]:
                return first_result["generations"][0]  # ✅ Extract biography text

        print("❌ Distilabel returned an empty response.")
        return ""

    except Exception as e:
        print(f"❌ OpenAI via Distilabel Error: {e}")
        return ""

# 🔥 Process Biographies Using OpenAI GPT-4 Turbo
for index, row in df.iterrows():
    # Convert JSON string to dictionary
    elites_json = row["synthetic_json"]
    elites_data = json.loads(elites_json)

    # Format the prompt
    prompt = """
    You are an expert historian and writer specializing in biographies of political elites (Vietnam or Russia).
    Your task is to generate a **detailed, Wikipedia-style biography** based on structured JSON data.

    ### **Example JSON Record & Biography**
    Below is an **example JSON record** and its corresponding **expected biography**:

    **Example JSON Record:**
    {
      "Name": "To Huy Rua",
      "Name_Other": [],
      "BirthYear": "1947",
      "DeathYear": "",
      "Birthplace": {"City": "Quang Xuong", "City_OtherName": [], "Region": "Thanh Hoa",  "Region_OtherName": []},
      "Ethnicity": [],
      "Education": [
        {"School": "Central Propaganda School", "OtherName": ["Academy of Journalism and Communication"], "Start": "1970", "End": "1974", "Level": "", "Field": "", "Location": []},
        {"School": "Hanoi University of Science", "OtherName": ["Vietnam National University, Hanoi"], "Start": "1975", "End": "1979", "Level": "University", "Field": "Mathematics", "Location": ["Hanoi"]},
        {"School": "Academy of Social Sciences", "OtherName": [], "Start": "1980", "End": "1986", "Level": "Doctorate", "Field": "Philosophy", "Location": ["USSR"]}
          ],
      "MilitaryTitle": [{"Title": "", "YearReceived": ""}],
      "Job": [
        {"Start": "1965", "End": "1970", "Position": "", "Organization": {"MainOrg": "Youth Volunteer Force", "MainOrg_OtherName": "", "SubOrg": "", "Unit": ""}, "Location": ["South"]},
        {"Start": "1974", "End": "1979", "Position": "Teaching Assistant", "Organization": {"MainOrg": "Central Propaganda School", "MainOrg_OtherName": "Academy of Journalism and Communication", "SubOrg": "", "Unit": ""}, "Location": []},
        {"Start": "1986", "End": "", "Position": "Deputy Head", "Organization": {"MainOrg": "Hanoi University of Science", "MainOrg_OtherName": "Vietnam National University, Hanoi", "SubOrg": "Faculty of Philisophy", "Unit": ""}, "Location": ["Hanoi"]},
        {"Start": "1990", "End": "1995", "Position": "Director", "Organization": {"MainOrg": "Central Propaganda School", "MainOrg_OtherName": "Academy of Journalism and Communication", "SubOrg": "", "Unit": ""}, "Location": []},
        {"Start": "1996", "End": "", "Position": "Deputy Director", "Organization": {"MainOrg": "Ho Chi Minh National Academy of Politics", "MainOrg_OtherName": "Ho Chi Minh National Academy of Politics and Public Administration", "SubOrg": "", "Unit": ""}, "Location": []},
        {"Start": "2000", "End": "", "Position": "Secretary", "Organization": {"MainOrg": "City Party Committee", "MainOrg_OtherName": "", "SubOrg": "", "Unit": ""}, "Location": ["Hai Phong"]},
        {"Start": "2003", "End": "", "Position": "Deputy Director", "Organization": {"MainOrg": "Ho Chi Minh National Academy of Politics and Public Administration", "MainOrg_OtherName": "", "SubOrg": "", "Unit": ""}, "Location": []},
        {"Start": "2005", "End": "", "Position": "Director", "Organization": {"MainOrg": "Ho Chi Minh National Academy of Politics and Public Administration", "MainOrg_OtherName": "Ho Chi Minh National Academy of Politics", "SubOrg": "", "Unit": ""}, "Location": []},
        {"Start": "May 2007", "End": "2010", "Position": "Chairman", "Organization": {"MainOrg": "CPV", "MainOrg_OtherName": "", "SubOrg": "Central Theoretical Council", "Unit": ""}, "Location": []},
        {"Start": "May 2007", "End": "2010", "Position": "Head", "Organization": {"MainOrg": "Communist Party of Vietnam", "MainOrg_OtherName": "CPV", "SubOrg": "Central Propaganda Department", "Unit": ""}, "Location": []}
        ],
      "Retired": [{"Date": "January 2016", "Detail": ["At the 12th National Congress (January 2016) he retired."]}],
      "Dismissed": [{"Date": "", "Detail": []}],
      "Resigned": [{"Date": "", "Detail": []}],
      "Arrested": [{"Date": "", "Detail": []}],
      "Exiled": [{"Date": "", "Detail": []}],
      "Killed": [{"Date": "", "Detail": []}],
      "FamilyMember": [{"Name": "Truong Tuyet Nhung", "Relation": "Wife"}, {"Name": "To Tu Ha", "Relation": "Son"}, {"Name": "To Huy Vu", "Relation": "Son"}, {"Name": "To Linh Huong", "Relation": "Daughter"}],
      "Colleague": [{"FirstMention_Year": "1999", "Name": ["Le Kha Phieu"]}, {"FirstMention_Year": "", "Name": ["Pham Van Vung"]}],
      "Rival": [{"FirstMention_Year": "", "Name": []}]
    }

    **Expected Biography:**
    To Huy Rua (born 1947 ) is a Vietnamese politician. He was a former member of the Politburo of the Communist Party of Vietnam (CPV), Head of the Central Organization Committee, former Head of the Central Propaganda Department. He has a Bachelor's degree in Mathematics and a Doctorate in Philosophy.
    Biography
    He was born on June 4, 1947 , in Quang Thai commune, Quang Xuong district, Thanh Hoa province .
    In 1965, he joined the Youth Volunteer Force to serve on the battlefield in the Southern region. In the early 1970, he was sent to study at the Central Propaganda School (now the Academy of Journalism and Communication), majoring in Philosophy. After graduating in 1974, he was retained as a teaching assistant at the Faculty of Philosophy at the school. During his time as a teaching assistant, he registered for additional studies (1975 – 1979) and earned a Bachelor's degree in Mathematics at Hanoi University of Science (now Vietnam National University, Hanoi).
    In 1980, he was appointed as a postgraduate student, majoring in Philosophy at the Academy of Social Sciences of the USSR, successfully defending his doctoral thesis . After graduating in 1986, he returned to Vietnam and was appointed Deputy Head of the Faculty of Philosophy, Hanoi University of Science (now Vietnam National University, Hanoi).
    From 1990 to 1995, he was elected Director of the Central Propaganda School (now the Academy of Journalism and Communication ). During this time, he was awarded the title of Meritorious Teacher by the Vietnamese State and the academic title of Associate Professor of Philosophy.
    In 1996, he was appointed Deputy Director of the Ho Chi Minh National Academy of Politics (now known as the Ho Chi Minh National Academy of Politics and Public Administration).
    At the end of 1999, he was appointed by his fellow countryman, General Secretary (of the CPV) Le Kha Phieu,  as Secretary of the Hai Phong City Party Committee and Secretary of the Hai Phong City Military Party Committee. He began this position in 2000.
    In 2003, he was transferred back to work as Deputy Director of the Ho Chi Minh National Academy of Politics and Public Administration.
    At the end of 2004, he was appointed Director of the Ho Chi Minh National Academy of Politics and Public Administration (also known as the Ho Chi Minh National Academy of Politics) and took charge of this position in 2005.
    From May 8, 2007 to 2010, To Huy Rua was assigned as Chairman of the Central Theoretical Council (of the CPV) and Head of the Central Propaganda Department of the Communist Party of Vietnam (CPV).
    At the 12th National Congress (January 2016) he retired.
    Family
    • His wife is Ms. Truong Tuyet Nhung, former Head of Training Department, Academy of Journalism and Propaganda.
    •	First son: To Tu Ha, Director of Noi Bai International Airport.
    •	Second son: To Huy Vu, currently Director of the International Cooperation Department of the State Bank of Vietnam.
    •	Youngest daughter: To Linh Huong, Chairwoman of the Board of Directors of Vinaconex - PVC Joint Stock Company.
    Controversial political activity
    To Huy Rua was reported to having sponsored the rise of Pham Van Vung to be a Navy Political Commissar. When he became Navy Political Commissar, Pham Van Vung revealed his true colors as a corrupt official, and was forced to hand over his position after 1 year in office.

    ---

    ### **Task: Now Generate a Biography for the Following JSON Record**

    """.strip() + "\n\n" + json.dumps(elites_data, indent=2) + "\n\n" + """

    Important Notes:
    - The biography should be **500-1100 words**, depending on the amount of available details of the JSON record.
    - Write the biography in a **neutral, Wikipedia-style format**, **without a References section**.
    - When writing the biography, include **all available JSON details**, such as education, job experiences, dismissal, retirement, titles, family members, colleagues, and rivals.
    - **DO NOT fabricate missing details**. If certain sections or JSON fields are empty, omit missing information naturally and avoid mentioning them explicitly.
    - **Enhance readability and coherence** while preserving facts.

        """

    # Add to batch
    prompts.append(prompt)
    batch_indices.append(index)

    # Process when batch is full or at last record
    if len(prompts) == BATCH_SIZE or index == len(df) - 1:
        try:
            print("\n📌 Processing batch...")

            openai_responses = []

            for i, p in enumerate(prompts):
                print(f"\n🔹 Generating biography {i+1}/{len(prompts)}...")

                openai_text = generate_with_openai(p)

                print(f"📌 OpenAI Output (First 200 chars): {openai_text[:200]}" if openai_text else "❌ OpenAI returned an empty response!")

                openai_responses.append(openai_text)

            # Store results in DataFrame
            for i, row_index in enumerate(batch_indices):
                df.loc[row_index, "syntetic_bio"] = openai_responses[i] if openai_responses[i] else ""

                if not openai_responses[i].strip():
                    failed_indices.append(row_index)

            print("\n📌 DataFrame Preview After Saving:")
            print(df.loc[batch_indices])

            # Free memory
            del openai_responses

            # Smart delay to avoid rate limits
            time.sleep(2)

            # Clear prompts and indices for next batch
            prompts, batch_indices = [], []

        except Exception as e:
            print(f"❌ Batch processing error: {e}")
            failed_indices.append(index)


# 🔥 Final confirmation
print("\n📌 Final DataFrame Preview:")
print(df.head())

# ⚠️ Log failed attempts
if failed_indices:
    print(f"⚠️ Failed to generate {len(failed_indices)} biographies. Retry these indices: {failed_indices}")

# 💾 Save to Excel
excel_path = "/content/drive/elitenet_synthetic_json_bio.xlsx"
df.to_excel(excel_path, index=False, engine="openpyxl")
print(f"✅ Updated Excel file saved: {excel_path}")



📌 Processing batch...

🔹 Generating biography 1/4...
📌 OpenAI Output (First 200 chars): ### Biography of Bui Thi An

Bui Thi An (1945–2020), also known as Tran Thi An, was a prominent Vietnamese civil engineer and politician, notable for her distinguished career within the Ministry of Tr

🔹 Generating biography 2/4...
📌 OpenAI Output (First 200 chars): ### Biography of Tran Thi Minh

Tran Thi Minh (1939–2016) was a prominent Vietnamese politician and military officer, known for her extensive service within Vietnam's Ministry of Interior. Throughout 

🔹 Generating biography 3/4...
📌 OpenAI Output (First 200 chars): ### Biography of Pham Van An

Pham Van An, also known by aliases Nguyen Thanh Quang and Ngo Ngoc Dung, is a former Vietnamese banker and politician born in 1951 in Vu Thu, Thai Binh province. He is re

🔹 Generating biography 4/4...
📌 OpenAI Output (First 200 chars): ### Biography of Tran Huu Cuong

**Tran Huu Cuong** (born 1949) is a retired Vietnamese politician known for h

#### Generator 2: Llama 3.3 70B

In [ ]:
# Load the Llama model
from distilabel.models.llms import TogetherLLM

llama_model = TogetherLLM(model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                         api_key="yourTogetherLLM-APIkey")

# Check if initialization is successful
if llama_model is None:
    print("❌ ERROR: Model initialization failed!")
else:
    print("✅ Model initialized successfully.")

# Check if the model loads successfully
try:
    llama_model.load()
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ ERROR: Model failed to load! \n{e}")

In [ ]:
import os
import json
import time
import pandas as pd
import torch
# from distilabel.models.llms.huggingface import InferenceEndpointsLLM  # Use Hugging Face API
# from distilabel.models.llms import TransformersLLM  # Uncomment if running locally
# from transformers import AutoTokenizer, AutoModelForCausalLM  # Uncomment for local

# Enable PyTorch memory optimizations
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("📌 Initial DataFrame Preview:")
print(df.head())

# 🚀 Alternative: Load Llama 3.3-70B locally (if you have a powerful GPU)
# llama_model_name = "meta-llama-3-70b"
# tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
# llama_model = AutoModelForCausalLM.from_pretrained(
#     llama_model_name,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

BATCH_SIZE = 4  # Adjust based on available VRAM
DEBUG = False

prompts, batch_indices, failed_indices = [], [], []

def safe_generate(model, prompt, max_tokens=1500, temp=0.6):
    """ Wrapper to handle API failures gracefully and extract text properly. """
    try:
        response = model.generate(
            [[{"role": "user", "content": prompt}]],
            max_new_tokens=max_tokens,
            temperature=temp
        )

        if isinstance(response, list) and len(response) > 0 and isinstance(response[0], dict):
            generations = response[0].get("generations", [])
            if isinstance(generations, list) and len(generations) > 0:
                return generations[0]  # Extract generated text
            else:
                print("❌ Empty `generations` list.")
        else:
            print("❌ Unexpected response format:", response)

        return ""  # Return empty string if extraction fails

    except Exception as e:
        print(f"❌ Generation error: {e}")
        return ""

# 🔥 Process Biographies Using Llama Only
for index, row in df.iterrows():
    elites_json = row["Input_JSON"]
    elites_data = json.loads(elites_json)  # Convert string to dictionary

    prompt = f"""
    You are an expert historian and writer specializing in biographies of Vietnam's political elites.
    Generate a detailed biography based on the following elite's JSON record:

    {json.dumps(elites_data, indent=2)}

    Important Notes:
    - The biography should be around 500 to 1000 words, depending on the JSON record's details.
    - Write the biography in a **neutral, Wikipedia-style format**, but **without a References section**.
    - Include **all available biographical details** from the JSON, such the elite's education, working experiences, dismissal, retirement, titles (if any), their family members, colleagues, rivals, and other details of their relationships (such as their family relation type or when they are first reported to work together and/or become rivals).
    This essentially means copy-paste the all the avaialble content in the JSON fields (if they are not empty) into your sentences and paragraphs when you write the biography.
    - **DO NOT fabricate missing details**. If certain information is unavailable, omit it naturally in the biography.
    - Your task is **not to alter the facts** but to enhance readability and coherence. When possible, you can also add in some contextual information outside of what is available in JSON.
    """

    prompts.append(prompt)
    batch_indices.append(index)

    if len(prompts) == BATCH_SIZE or index == len(df) - 1:
        try:
            print("\n📌 Processing batch...")

            llama_responses = []

            for i, p in enumerate(prompts):
                print(f"\n🔹 Generating biography {i+1}/{len(prompts)}...")

                llama_text = safe_generate(llama_model, p)

                print(f"📌 Llama 3.3 Output (First 200 chars): {llama_text[:200]}" if llama_text else "❌ Llama 3.3 returned an empty response!")

                llama_responses.append(llama_text)

            # ✅ Store results in DataFrame
            for i, llama_bio in enumerate(llama_responses):
                df.loc[batch_indices[i], "Llama_3.3_Bio"] = llama_bio.strip() if llama_bio else ""

                if not llama_bio.strip():
                    failed_indices.append(batch_indices[i])

            print("\n📌 DataFrame Preview After Saving:")
            print(df.loc[batch_indices])

            # Free memory after each batch
            del llama_responses

            # ⏳ Smart delay to avoid rate limits
            time.sleep(3)

        except Exception as e:
            print(f"❌ Batch processing error: {e}")
            torch.cuda.empty_cache()

        # Clear prompts and indices for next batch
        prompts, batch_indices = [], []

# 🔥 Final confirmation
print("\n📌 Final DataFrame Preview:")
print(df.head())

# ⚠️ Log failed attempts
if failed_indices:
    print(f"⚠️ Failed to generate {len(failed_indices)} biographies. Retry these indices: {failed_indices}")

# 💾 Save to Excel
df.to_excel(excel_path, index=False, engine="openpyxl")
print(f"✅ Updated Excel file saved: {excel_path}")


📌 Initial DataFrame Preview:
                                          Input_JSON Mistral_Bio Llama_3.3_Bio
0  {"Name": "Bui Danh Luu", "Name_Other": ["Quoc ...                          
1  {"Name": "Bui Thien Ngo", "Name_Other": [], "B...                          
2  {"Name": "Cao Si Kiem", "Name_Other": ["Cao Sy...                          
3  {"Name": "Chu Tuan Nha", "Name_Other": [], "Bi...                          
4  {"Name": "Dam Quang Trung", "Name_Other": ["Da...                          

📌 Processing batch...

🔹 Generating biography 1/4...
📌 Llama 3.3 Output (First 200 chars): Bui Danh Luu, also known as Quoc Linh, was a prominent Vietnamese political figure born in 1935 in Thanh Thuy, Phu Tho. Little is known about his early life, including his ethnicity. However, it is re

🔹 Generating biography 2/4...
📌 Llama 3.3 Output (First 200 chars): Bui Thien Ngo was a Vietnamese politician born in 1929 in Saigon, also known as Ho Chi Minh City. There is no available information on 

In [ ]:
# 💾 Save to Excel
df.to_excel(excel_path, index=False, engine="openpyxl")
print(f"✅ Updated Excel file saved: {excel_path}")

✅ Updated Excel file saved: /content/drive/MyDrive/EliteNet_SyntheticData_2025/elitenet_synthetic_2llm.xlsx


### Synthetic Paraphrased Biographies (From Original & Factually Correct JSONs)


In [ ]:
import pandas as pd
import torch

# Load dataset
file_path = "/content/drive/llm_train_human_augmented.xlsx"  # Ensure correct path
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Ensure all input text is string
df["input"] = df["input"].fillna("").astype(str)

In [ ]:
# Load the aggregator
from distilabel.models.llms import TogetherLLM

llama_model = TogetherLLM(model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                         api_key="yourTogetherLLM-APIkey")

# Check if initialization is successful
if llama_model is None:
    print("❌ ERROR: Model initialization failed!")
else:
    print("✅ Model initialized successfully.")

# Check if the model loads successfully
try:
    llama_model.load()
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ ERROR: Model failed to load! \n{e}")

✅ Model initialized successfully.
✅ Model loaded successfully.


In [ ]:
import os
import json
import time
import torch

# Enable PyTorch memory optimizations
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("📌 Initial DataFrame Preview:")
print(df.head())

BATCH_SIZE = 4  # Adjust based on VRAM and API rate limits
DEBUG = False

prompts, batch_indices, failed_indices = [], [], []

def safe_generate(model, prompt, max_tokens=2000, temp=0.65):
    """Wrapper to handle API failures gracefully and extract text properly."""
    try:
        response = model.generate(
            [[{"role": "user", "content": prompt}]],
            max_new_tokens=max_tokens,
            temperature=temp
        )

        if isinstance(response, list) and len(response) > 0 and isinstance(response[0], dict):
            generations = response[0].get("generations", [])
            if isinstance(generations, list) and len(generations) > 0:
                return generations[0]  # Extract generated text
            else:
                print("❌ Empty `generations` list.")
        else:
            print("❌ Unexpected response format:", response)

        return ""  # Return empty string if extraction fails

    except Exception as e:
        print(f"❌ Generation error: {e}")
        return ""

# 🔥 Process Biographies Using LLaMA 3.3-70B
for index, row in df.iterrows():
    original_bio = row["input"]

    if not original_bio.strip():
        continue  # Skip empty biographies

    prompt = f"""
    You are an expert historian and writer. Your task is to paraphrase the an elite's biography while keeping **all factual details** intact.

    ### Two examples of **a few paragraphs of a Biography** and **their paraphrased versions**:
    **Biography 1:**
    In 1948, Pham Van Dong began his tenure as Vice Premier in the Government Office, a position he held until 1955. Concurrently, he served as Minister of Foreign Affairs from 1955 to 1960.
    He also held the position of Premier (equivalent to now Prime Minister) in the Government Office from 1955 to 1959. From 1960 to 1980, he continued to serve as Premier, this time in the Council of Government. This made Pham Van Dong the Prime Minister/Premier who served in the government throughout two consecutive wars against France and America.
    In 1981, the Government was renamed into Council of Ministers. Thus, Pham Van Dong's continued his tenure as Chairman of the Council of Ministers from 1981 to 1987. During this time, Pham Van Dong and Nguyen Van Linh exrted all possible influence to help film director Tran Van Thuy release the movie "Hanoi in Whose Eyes", despite being initially banned by other high-ranking politicians.

    **Paraphrase 1:**
    From 1948 to 1955, Pham Van Dong served as Vice President of the Government (or Government Office). From 1955 to 1960, he was also Minister of Foreign Affairs.
    He then ascended to became the top leader of the government, serving throughout two consecutive wars against France and America as Premier (eqquivalent to now Prime Minister) of the Government Office (from 1955 to 1959), Premier of the Council of Government (from 1960 to 1980), and Chairman of the Council of Ministers (from 1981 to 1987). Also during this time, Pham Van Dong, along with Nguyen Van Linh, did everything in his power to help film director Tran Van Thuy release the movie "Hanoi in Whose Eyes", despite previously being banned by other high-ranking government officials.

    **Biography 2:**
    From 1963, Pham Gia Khiem studied at Hanoi University of Science and Technology, also known as Hanoi University of Technology. In 1967, he graduated with a bachelor's degree in Mechanical Engineering and was assigned as an instructor at the University of Industrial Technology, Thai Nguyen University, from 1967 to 1970. He later pursued his Doctorate in Metallurgy from 1971 to 1975 in Czechoslovakia and successfully defended his doctoral thesis in this field.

    **Paraphrase 2:**
    1963 - 1967: Pham Gia Khiem studied at and then graduated from Hanoi University of Science and Technology, also known as Hanoi University of Technology, with a bachelor's degree in Mechanical Engineering.
    1967 - 1970: He worked as an instructor at the University of Industrial Technology, Thai Nguyen University.
    1971 - 1975: He studied at the Doctorate level and successfully defended his thesis in the field of Metallurgy in Czechoslovakia.


    ### **Task: Now Generate a Biography for the Following JSON Record**
    **Original Biography:**
    {original_bio}

    **Guidelines:**
    - Rewrite the biography using different vocabulary, sentence structures, or paragraph structures.
    - Do **not** add or remove any facts. Do **not** alter facts either. These facts (who, what, when, where) include people, actions, organizations, jobs, education, time, and locations.
    - Maintain a **neutral, Wikipedia-style tone**.
    - Improve coherence and readability.

    Provide only the paraphrased biography as output.
    """

    prompts.append(prompt)
    batch_indices.append(index)

    if len(prompts) == BATCH_SIZE or index == len(df) - 1:
        try:
            print("\n📌 Processing batch...")

            llama_responses = []

            for i, p in enumerate(prompts):
                print(f"\n🔹 Generating paraphrased biography {i+1}/{len(prompts)}...")

                llama_text = safe_generate(llama_model, p)

                print(f"📌 Llama 3.3 Output (First 200 chars): {llama_text[:200]}" if llama_text else "❌ Llama 3.3 returned an empty response!")

                llama_responses.append(llama_text)

            # ✅ Store results in DataFrame
            for i, llama_bio in enumerate(llama_responses):
                df.loc[batch_indices[i], "synthetic_input"] = llama_bio.strip() if llama_bio else ""

                if not llama_bio.strip():
                    failed_indices.append(batch_indices[i])

            print("\n📌 DataFrame Preview After Saving:")
            print(df.loc[batch_indices])

            # Free memory after each batch
            del llama_responses

            # ⏳ Smart delay to avoid rate limits
            time.sleep(3)

        except Exception as e:
            print(f"❌ Batch processing error: {e}")
            torch.cuda.empty_cache()

        # Clear prompts and indices for next batch
        prompts, batch_indices = [], []

# 🔥 Final confirmation
print("\n📌 Final DataFrame Preview:")
print(df.head())

# ⚠️ Log failed attempts
if failed_indices:
    print(f"⚠️ Failed to generate {len(failed_indices)} paraphrases. Retry these indices: {failed_indices}")

# 💾 Save to Excel
output_file = "/content/drive/elitenet_synthetic_bio_paraphrased.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")
print(f"✅ Updated Excel file saved: {output_file}")


📌 Initial DataFrame Preview:
                                         instruction  \
0  You are an expert historian and political scie...   
1  You are an expert historian and political scie...   
2  You are an expert historian and political scie...   
3  You are an expert historian and political scie...   
4  You are an expert historian and political scie...   

                                               input  \
0  Bùi Danh Lưu, also known by his other name Quo...   
1  Bui Thien Ngo\n================\n\nBui Thien N...   
2  Cao Si Kiem is a Vietnamese economist and poli...   
3  Chu Tuan Nha is a former Vietnamese politician...   
4  Dam Quang Trung (1921-1995) was a general in t...   

                                              output  
0  {"Name": "Bui Danh Luu", "Name_Other": ["Quoc ...  
1  {"Name": "Bui Thien Ngo", "BirthYear": "1929",...  
2  {"Name": "Cao Si Kiem", "Name_Other": ["Cao Sy...  
3  {"Name": "Chu Tuan Nha", "BirthYear": "1939", ...  
4  {"Name": "Dam Quang

In [ ]:
# List of failed indices
retry_indices = [44, 46, 49, 51, 52, 54, 56, 62, 65, 67, 68, 69, 73, 74, 75, 77, 78, 79,
                  80, 81, 83, 84, 85, 86, 87, 90, 91, 92, 93, 94, 96, 98, 99, 100]

# Extract the subset of the dataset containing only the failed rows
df_failed = df.loc[retry_indices].copy()

print(f"📌 Extracted {len(df_failed)} failed biographies for re-processing.")

📌 Extracted 34 failed biographies for re-processing.


In [ ]:
import os
import json
import time
import torch

# Enable PyTorch memory optimizations
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("📌 Initial DataFrame Preview:")
print(df_failed.head())

BATCH_SIZE = 3  # Adjust based on VRAM and API rate limits
DEBUG = False

prompts, batch_indices, failed_indices = [], [], []

def safe_generate(model, prompt, max_tokens=2000, temp=0.65):
    """Wrapper to handle API failures gracefully and extract text properly."""
    try:
        response = model.generate(
            [[{"role": "user", "content": prompt}]],
            max_new_tokens=max_tokens,
            temperature=temp
        )

        if isinstance(response, list) and len(response) > 0 and isinstance(response[0], dict):
            generations = response[0].get("generations", [])
            if isinstance(generations, list) and len(generations) > 0:
                return generations[0]  # Extract generated text
            else:
                print("❌ Empty `generations` list.")
        else:
            print("❌ Unexpected response format:", response)

        return ""  # Return empty string if extraction fails

    except Exception as e:
        print(f"❌ Generation error: {e}")
        return ""

# 🔥 Process Biographies Using LLaMA 3.3-70B
for index, row in df_failed.iterrows():
    retry_bio = row["input"]

    if not retry_bio.strip():
        continue  # Skip empty biographies

    prompt = f"""
    You are an expert historian and writer. Your task is to paraphrase the an elite's biography while keeping **all factual details** intact.

    ### Two examples of **a few paragraphs of a Biography** and **their paraphrased versions**:
    **Biography 1:**
    In 1948, Pham Van Dong began his tenure as Vice Premier in the Government Office, a position he held until 1955. Concurrently, he served as Minister of Foreign Affairs from 1955 to 1960.
    He also held the position of Premier (equivalent to now Prime Minister) in the Government Office from 1955 to 1959. From 1960 to 1980, he continued to serve as Premier, this time in the Council of Government. This made Pham Van Dong the Prime Minister/Premier who served in the government throughout two consecutive wars against France and America.
    In 1981, the Government was renamed into Council of Ministers. Thus, Pham Van Dong's continued his tenure as Chairman of the Council of Ministers from 1981 to 1987. During this time, Pham Van Dong and Nguyen Van Linh exrted all possible influence to help film director Tran Van Thuy release the movie "Hanoi in Whose Eyes", despite being initially banned by other high-ranking politicians.

    **Paraphrase 1:**
    From 1948 to 1955, Pham Van Dong served as Vice President of the Government (or Government Office). From 1955 to 1960, he was also Minister of Foreign Affairs.
    He then ascended to became the top leader of the government, serving throughout two consecutive wars against France and America as Premier (eqquivalent to now Prime Minister) of the Government Office (from 1955 to 1959), Premier of the Council of Government (from 1960 to 1980), and Chairman of the Council of Ministers (from 1981 to 1987). Also during this time, Pham Van Dong, along with Nguyen Van Linh, did everything in his power to help film director Tran Van Thuy release the movie "Hanoi in Whose Eyes", despite previously being banned by other high-ranking government officials.

    **Biography 2:**
    From 1963, Pham Gia Khiem studied at Hanoi University of Science and Technology, also known as Hanoi University of Technology. In 1967, he graduated with a bachelor's degree in Mechanical Engineering and was assigned as an instructor at the University of Industrial Technology, Thai Nguyen University, from 1967 to 1970. He later pursued his Doctorate in Metallurgy from 1971 to 1975 in Czechoslovakia and successfully defended his doctoral thesis in this field.

    **Paraphrase 2:**
    1963 - 1967: Pham Gia Khiem studied at and then graduated from Hanoi University of Science and Technology, also known as Hanoi University of Technology, with a bachelor's degree in Mechanical Engineering.
    1967 - 1970: He worked as an instructor at the University of Industrial Technology, Thai Nguyen University.
    1971 - 1975: He studied at the Doctorate level and successfully defended his thesis in the field of Metallurgy in Czechoslovakia.


    ### **Task: Now Generate a Biography for the Following JSON Record**
    **Original Biography:**
    {retry_bio}

    **Guidelines:**
    - Rewrite the biography using different vocabulary, sentence structures, or paragraph structures.
    - Do **not** add or remove any facts. Do **not** alter facts either. These facts (who, what, when, where) include people, actions, organizations, jobs, education, time, and locations.
    - Maintain a **neutral, Wikipedia-style tone**.
    - Improve coherence and readability.

    Provide only the paraphrased biography as output.
    """

    prompts.append(prompt)
    batch_indices.append(index)

    if len(prompts) == BATCH_SIZE or index == len(df_failed) - 1:
        try:
            print("\n📌 Processing batch...")

            llama_responses = []

            for i, p in enumerate(prompts):
                print(f"\n🔹 Generating paraphrased biography {i+1}/{len(prompts)}...")

                llama_text = safe_generate(llama_model, p)

                print(f"📌 Llama 3.3 Output (First 200 chars): {llama_text[:200]}" if llama_text else "❌ Llama 3.3 returned an empty response!")

                llama_responses.append(llama_text)

            # ✅ Store results in DataFrame
            for i, llama_bio in enumerate(llama_responses):
                df_failed.loc[batch_indices[i], "synthetic_input"] = llama_bio.strip() if llama_bio else ""

                if not llama_bio.strip():
                    failed_indices.append(batch_indices[i])

            print("\n📌 DataFrame Preview After Saving:")
            print(df_failed.loc[batch_indices])

            # Free memory after each batch
            del llama_responses

            # ⏳ Smart delay to avoid rate limits
            time.sleep(3)

        except Exception as e:
            print(f"❌ Batch processing error: {e}")
            torch.cuda.empty_cache()

        # Clear prompts and indices for next batch
        prompts, batch_indices = [], []

# 🔥 Final confirmation
print("\n📌 Final DataFrame Preview:")
print(df_failed.head())

# ⚠️ Log failed attempts
if failed_indices:
    print(f"⚠️ Failed to generate {len(failed_indices)} paraphrases. Retry these indices: {failed_indices}")

📌 Initial DataFrame Preview:
                                          instruction  \
44  You are an expert historian and political scie...   
46  You are an expert historian and political scie...   
49  You are an expert historian and political scie...   
51  You are an expert historian and political scie...   
52  You are an expert historian and political scie...   

                                                input  \
44  Nguyen Ky Cam is a Vietnamese political figure...   
46  Nguyen Manh Kiem (1938–2020) was a Vietnamese ...   
49  Nguyen Quyet (birth name: Nguyen Tien Van ) (A...   
51  Nguyen Tan Dung (born November 17, 1949 in Ca ...   
52  Nguyen Tan Trinh (December 20, 1936 - December...   

                                               output synthetic_input  
44                          {"Name": "Nguyen Ky Cam"}                  
46  {"Name": "Nguyen Manh Kiem", "BirthYear": "193...                  
49  {"Name": "Nguyen Quyet", "Name_Other": ["Nguye...                

In [ ]:
# ✅ Merge the new successful paraphrases back into the main dataset
df.loc[df_failed.index, "synthetic_input"] = df_failed["synthetic_input"]

# 🔥 Final confirmation
print("\n📌 Final DataFrame Preview After Merging Reprocessed Entries:")
print(df.head())


📌 Final DataFrame Preview After Merging Reprocessed Entries:
                                         instruction  \
0  You are an expert historian and political scie...   
1  You are an expert historian and political scie...   
2  You are an expert historian and political scie...   
3  You are an expert historian and political scie...   
4  You are an expert historian and political scie...   

                                               input  \
0  Bùi Danh Lưu, also known by his other name Quo...   
1  Bui Thien Ngo\n================\n\nBui Thien N...   
2  Cao Si Kiem is a Vietnamese economist and poli...   
3  Chu Tuan Nha is a former Vietnamese politician...   
4  Dam Quang Trung (1921-1995) was a general in t...   

                                              output  \
0  {"Name": "Bui Danh Luu", "Name_Other": ["Quoc ...   
1  {"Name": "Bui Thien Ngo", "BirthYear": "1929",...   
2  {"Name": "Cao Si Kiem", "Name_Other": ["Cao Sy...   
3  {"Name": "Chu Tuan Nha", "BirthYear":

In [ ]:
# 💾 Save to Excel
output_file = "/content/drive/elitenet_synthetic_bio_para_final.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")
print(f"✅ Fully updated Excel file saved: {output_file}")

✅ Fully updated Excel file saved: /content/drive/MyDrive/EliteNet_SyntheticData_2025/elitenet_synthetic_bio_para_final.xlsx


In [ ]:
### Save synthetic biographies to a text file for ease of manipulation ###

# Extract synthetic biographies from the 'synthetic_input' column
synthetic_biographies = df["synthetic_input"].dropna().astype(str).tolist()

text_output_path = "/content/drive/elitenet_synthetic_bio_para_final.txt"
# Save to a text file, each biography separated by a newline
with open(text_output_path, "w", encoding="utf-8") as f:
    for bio in synthetic_biographies:
        f.write(bio + "\n\n")  # Add extra newline for readability

print(f"✅ Synthetic biographies saved to: {text_output_path}")

✅ Synthetic biographies saved to: /content/drive/MyDrive/EliteNet_SyntheticData_2025/elitenet_synthetic_bio_para_final.txt
